# One-Dimension Line Search 一维线搜索

## 问题描述

Consider a rectangular wing of span $b$ and chord $c$. Its planform area is thus $ S=b c $ and its aspect ratio is $ A = b^{2} / S $. The drag of this wing can be approximated as,
$$
C_{D}=k C_{f} \frac{S_{w e t}}{S}+\frac{C_{L}^{2}}{\pi A e}
$$
The first term corresponds to the parasite drag. $ C_{f} $ is the skin friction coefficient, which for a fully turbulent boundary layer can be approximated as,
$$
C_{f}=\frac{0.074}{R e^{0.2}}
$$
Here, the Reynolds number $ (R e=\rho V c / \mu) $ is based on the wing chord. $ k $ is the form factor, which accounts for the effects of pressure drag.

The second term in Equation (1) is the induced drag, where $ e $ is the Oswald efficiency factor. The lift coefficient $ C_{L} $ and the wing planform area $ S $ are to be kept constant. The values for all the constants are listed in Table 1. 

1. Write the total drag coefficient as a function of $ A $.
2. Minimize $ C_{D} $ with respect to $ A $ using:   
    (a) The golden section method  
    (b) A line search method that satisfies sufficient decrease. (Bonus: A line search that satisfies the strong Wolfe conditions.)  
    Converge the solutions to 6 significant digits.  
3. Discuss the relative performance of these two methods. Try different starting points/intervals and compare convergence rates, number of iterations and any other metrics you find suitable.

\begin{array}{lrll}
\text { Quantity } & \text { Value } & \text { Units } & \text { Description } \\
\hline \rho & 1.23 & \mathrm{~kg} / \mathrm{m}^{3} & \text { density of air } \\
\mu & 17.8 \times 10^{-6} & \mathrm{~kg} /(\mathrm{m} \mathrm{sec}) & \text { viscosity of air } \\
V & 35 & \mathrm{~m} / \mathrm{s} & \text { airspeed } \\
S & 11.8 & \mathrm{~m}^{2} & \text { planform area } \\
S_{\text {wet }} & 2.05 S & \mathrm{~m}^{2} & \text { wing wetted area } \\
k & 1.2 & & \text { form factor } \\
C_{L} & 0.3 & & \text { lift coefficient } \\
e & 0.96 & & \text { Oswald efficiency factor } \\
\hline
\end{array}


In [3]:
import numpy as np

def Cal_Re(c, rho=1.23, V=35.0, mu=17.8e-6):
    """Calculate Reynolds number."""
    return rho*V*c/mu

def Cal_Cf(Re):
    """Calculate the skin friction coefficient"""
    return 0.074/(Re**0.2)

def Cal_Cd(A, k=1.2, S=11.8, S_ratio=2.05, Cl=0.3, e=0.96):
    """Calculate the total drag coefficient as the function of A"""
    b = np.sqrt(A*S)
    c = S/b
    Re = Cal_Re(c)
    Cf = Cal_Cf(Re)
    return k*Cf*S_ratio+Cl**2/(np.pi*A*e)

## The Golden Section Method 黄金切割法

In [4]:
class GoldenSectionMethod():
    """Golden Section Method"""
    def __init__(self, function, x_min=1.0, x_max=100.0, tau=0.618, error = 1e-6):
        self.x_max = x_max
        self.x_min = x_min
        self.function = function
        self.tau = tau
        self.error = error

    def Search(self, iter_max=2000):
        # init step
        b = self.x_max
        a = self.x_min
        n = max( a + (b-a)*self.tau, b - (b-a)*self.tau )
        m = min( a + (b-a)*self.tau, b - (b-a)*self.tau )
        
        # search
        iteration = 1
        min_index_old = 100000
        while(iteration<=iter_max):
            if( self.function(m) < self.function(n)):
                min_index = m
                b = n
            elif( self.function(m) > self.function(n)):
                min_index = n 
                a = m
            else:
                min_index = (m+n)/2.0
                a = m
                b = n
            n = max( a + (b-a)*self.tau, b - (b-a)*self.tau )
            m = min( a + (b-a)*self.tau, b - (b-a)*self.tau ) 

            if(abs(min_index - min_index_old)<=self.error):
                print("The accuracy has reached "+str(self.error))
                print('Iteration = '+str(iteration))
                break

            min_index_old = min_index
            iteration += 1

        return min_index, self.function(min_index)

Golden = GoldenSectionMethod(function=Cal_Cd)
A_min, Cd_min = Golden.Search()
print(A_min,Cd_min)

The accuracy has reached 1e-06
Iteration = 22
28.394733234617927 0.011560688987335527


## A line search that satisfies the strong Wolfe conditions.

对于一维搜索问题，定义：
$$
\phi (A_{k} + \alpha) = C_{d} = k C_{f} \frac{S_{w e t}}{S}+\frac{C_{L}^{2}}{\pi (A_{k} + \alpha) e}
$$
其中，
$$
C_{f} = \frac{0.074}{R e^{0.2}} \\
R e = \frac{\rho V c }{\mu} = \frac{\rho V \sqrt{S} }{\mu} \frac{1}{\sqrt{A_{k} + \alpha}}
$$
则函数$\phi$的一阶导数为：
$$
\phi^{'} (\alpha) = \frac{\mathrm{d} C_{d}}{\mathrm{d} \alpha} 
= k \frac{S_{w e t}}{S} \frac{\mathrm{d} C_{f}}{\mathrm{d} \alpha} - \frac{C_{L}^{2}}{\pi (A_{k} + \alpha)^{2} e}
$$

其中，
$$
\frac{\mathrm{d} C_{f}}{\mathrm{d} \alpha} = - \frac{0.2 \times 0.074}{R e^{1.2}} \frac{\mathrm{d} Re}{\mathrm{d} \alpha}
= \frac{0.2 \times 0.074}{R e^{1.2}} \frac{ \rho V \sqrt{S}}{\mu} (0.5*\frac{1}{(A_{k} + \alpha)^{1.5}})
$$


In [9]:
def grad_Cd(A, k=1.2, S=11.8, S_ratio=2.05, Cl=0.3, e=0.96, rho=1.23, V=35.0, mu=17.8e-6):
    """Calculate the gradient of Cd as the function of A"""
    b = np.sqrt(A*S)
    c = S/b
    Re = Cal_Re(c)
    phi1 = k*S_ratio*(0.2*0.074)/(Re**1.2)*rho*V*np.sqrt(S)/mu*0.5*(A**-1.5)
    phi2 = Cl**2/(np.pi * A**2 * e)

    return phi1 - phi2

class WolfeConditionsSearch():
    """A line search that satisfies the strong Wolfe conditions."""
    def __init__(self, function, grad_function, x_min=1.0, x_max=100.0, mu1=1e-4, mu2=0.9):
        self.x_max = x_max
        self.x_min = x_min
        self.function = function
        self.grad_function = grad_function
        self.mu1 = mu1
        self.mu2 = mu2
    
    def lineSearch(self, init_x=float('inf'), a_step = 1 , iter_max=2000):
        if(init_x == float('inf')):
            #init_x = np.random.uniform(self.x_min, self.x_max)
            init_x = self.x_min
        #print("The start points is "+str(init_x))
        
        # Search the space of a
        a_best = 0
        a1 = 0
        a2 = a1 + a_step
        find = False    
        while(True):
            # wheather the (x+a) has reached the max of x
            if((init_x+a2)>self.x_max):
                a_min = a1
                a_max = self.x_max
                break
            if(self.function(init_x + a2) > self.function(init_x) + self.mu1*a2*self.grad_function(init_x)):
                a_min = a1
                a_max = a2 
                break
            if(abs(self.grad_function(init_x + a2)) < -self.mu2*self.grad_function(init_x) and self.grad_function(init_x + a2)<0):
                a_best = a2
                find = True 
                break
            if(self.grad_function(init_x + a2)>0):
                a_min = a1
                a_max = a2 
                break
            a1 = a2
            a2 += a_step

        if(find):
            return a_best

        if(a_max == self.x_max):
            #print("There is no min points 101")
            return 101
            
        # decrease the space of a
        iteration = 1
        while(iteration<iter_max):  
            a1 = a_min
            a2 = (a_min + a_max) / 2.0
            if(self.function(init_x + a2) > self.function(init_x)+self.mu1*a2*self.grad_function(init_x)):
                a_max = a2 
                continue
            if(abs(self.grad_function(init_x + a2)) < abs(self.mu2*self.grad_function(init_x)) and self.grad_function(init_x + a2)<0):
                a_best = a2
                find = True 
                break
            if(self.grad_function(init_x + a2)*(a_max - a_min)>0):
                a_max = a2 
            elif(self.grad_function(init_x + a2)*(a_max - a_min)<0):
                a_min = a2
            iteration += 1
        
        if(find):
            return a_best
        else:
            #print("There is no min point 202")
            return 202

init_x = 0.1
i = 1
error = 10000
while( error > 1e-6 and i<1000):
    wolfe = WolfeConditionsSearch(Cal_Cd, grad_Cd)
    a = wolfe.lineSearch(init_x=init_x,a_step=1, iter_max=1000)
    if(a == 101 or a == 202):
        break
    #error = abs(Cal_Cd(init_x+a)-Cal_Cd(init_x))
    error = a
    init_x += a 
    i +=1
print(init_x, Cal_Cd(init_x))       

28.394247627258302 0.011560688987166721
